### 2. Calcular el número de 1-pixeles

In [ ]:
import plotly.graph_objects as go
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage
import math
import pandas as pd

names = ['apple-1.png', 'children-1.png', 'cup-1.png', 'device3-1.png', 
         'device4-1.png', 'Heart-1.png', 'personal_car-1.png', 'ray-1.png', 'sea_snake-1.png','turtle-1.png']

images = []
 
# Cargar y guardar las imagenes binarias
for name in names:
    fullName = 'images/' + name
    img = np.array(cv2.imread(fullName, cv2.IMREAD_GRAYSCALE))/255
    images.append(img.astype(int))

def showImages(images):
    fig,axs = plt.subplots(2,5,figsize=(25,10))
    for i,img in enumerate(images):
        row = i//5
        col = i%5
        axs[row,col].imshow(img,cmap = "Greys_r")
        axs[row,col].axis("off")
    plt.show()
showImages(images)


In [ ]:
resized_images = []
for image in images:
    resized_image = np.array(skimage.transform.resize(image,(100,100)))
    resized_image[resized_image != 0] = 1
    resized_images.append(resized_image)

showImages(resized_images)

In [ ]:
def count_one_pixels(image):
    return np.count_nonzero(image)

n_pixels = []
for image in resized_images:
    n_pixels.append(count_one_pixels(image))

fig = go.Figure(data=[go.Table(header=dict(values=['Imagen', 'Cantidad de 1-píxeles'], line_color='darkslategray',
                fill_color='gray', font=dict(color='white', size=12)),
                cells=dict(values=[names, n_pixels], line_color='darkslategray',
                fill_color='lightgray', font=dict(color='black', size=12)))])
fig.show()

count_one_pixels(resized_images)

### 3. Transformaciones de escala

In [ ]:
def calculate_center_of_mass(image):
    M = np.sum(image)
    x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
    Xcm = np.sum(x * image) / M
    Ycm = np.sum(y * image) / M
    return int(np.floor(Xcm)), int(np.floor(Ycm))

In [ ]:
def scale_object(image, center_of_mass, scale_factor):
    # Crear matriz de coordenadas
    coords = np.array(np.where(image)).T
    
    # Restar centro de masa a coordenadas
    coords_centered = coords - center_of_mass
    
    # Escalar coordenadas
    scaled_coords = coords_centered * scale_factor
    
    # Sumar centro de masa a coordenadas escaladas
    scaled_coords += center_of_mass
    
    # Crear imagen vacía para la imagen escalada
    scaled_image = np.zeros_like(image)
    
    # Convertir coordenadas escaladas en índices de imagen
    indices = np.round(scaled_coords).astype(int)
    
    # Asegurarse de que los índices estén dentro de los límites de la imagen
    indices[:, 0] = np.clip(indices[:, 0], 0, image.shape[1]-1)
    indices[:, 1] = np.clip(indices[:, 1], 0, image.shape[0]-1)
    
    # Establecer píxeles en la imagen escalada
    scaled_image[indices[:, 0], indices[:, 1]] = 1

    scaled_image[scaled_image != 0] = 1
    
    return scaled_image

In [ ]:
scaled_images = []

min = np.min(n_pixels)
for i in range(len(resized_images)):
   factor = math.sqrt(min / n_pixels[i])
   scaled_images.append(scale_object(resized_images[i], calculate_center_of_mass(resized_images[i]), factor))

for image in scaled_images:
   print(np.count_nonzero(image))

showImages(scaled_images)

### 4. Invariante de escala siguiente, antes y después del escalamiento con p,q = 0,1 y 2

$$
\mu_{pq} = \sum_{x=1}^{M}\sum_{y=1}^{N} (x - x_{cm})^p (y - y_{cm})^q
$$

$$
\eta_{pq} = \frac{\mu_{pq}}{\mu_{00}^{\frac{p + q}{2} + 1}}
$$

In [ ]:
def central_moment(image, p, q):
    x_cm, y_cm = calculate_center_of_mass(image)
    sum = 0
    for Y in range(image.shape[0]):
        for X in range(image.shape[1]):
            if (image[Y, X] != 0):
                sum += (X - x_cm) ** p * (Y - y_cm) ** q
    return sum

def normalized_moments(image, p, q):
    mu_pq = central_moment(image, p, q)
    mu_00 = central_moment(image, 0, 0)
    eta_pq = mu_pq / (mu_00**((p+q)/2 + 1))
    return eta_pq

for i in range(len(resized_images)):
    print('Imagen original', i + 1)
    for p in range(0, 3):
        q_values = []
        for q in range(0, 3):
            result = normalized_moments(resized_images[i], p, q)
            q_values.append(result)
        print(q_values)
    print('Imagen escalada', i + 1)
    for p in range(0, 3):
        q_values = []
        for q in range(0, 3):
            result = normalized_moments(scaled_images[i], p, q)
            q_values.append(result)
        print(q_values)
    print()

### 5. Obtener gráficos de las celdas (cuadrangulares) de resolución de los 1-pixeles

In [ ]:
for image in scaled_images:
    binary_image = image

    # Configurar el tamaño de la figura y la cuadrícula
    fig, ax = plt.subplots(figsize=(20,20))
    ax.set_xticks(np.arange(-0.5, binary_image.shape[1], 1))
    ax.set_yticks(np.arange(-0.5, binary_image.shape[0], 1))
    ax.grid(which='both', color='black', linestyle='-', linewidth=1)

    # Configurar la etiqueta para los ejes x e y
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # Mostrar la imagen binaria en la cuadrícula
    ax.imshow(binary_image, cmap='gray', interpolation='nearest')

    plt.show()

### 6. Obtener el contorno de cada uno de los objetos binarios, considerando la vecindad-8

In [ ]:
def get_border(image):
    binary_image = image

    border_image = np.zeros_like(binary_image)

    height, width = binary_image.shape
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            neighbors = [binary_image[i-1, j-1], binary_image[i-1, j], binary_image[i-1, j+1],
                        binary_image[i, j-1], binary_image[i, j+1],
                        binary_image[i+1, j-1], binary_image[i+1, j], binary_image[i+1, j+1]]
            if binary_image[i, j] == 1 and 0 in neighbors:
                border_image[i, j] = 1

    return border_image

bordered_images = []

for image in scaled_images:
    bordered_images.append(get_border(image))

showImages(bordered_images)

### 7. Calcular el centro de masa (𝑥𝑐𝑚, 𝑦𝑐𝑚) de cada uno de los objetos binarios. Trasladar la imagen original a una posición diferente y calcular los momentos centrales para p,q = 0,1 y 2

In [ ]:
for image in scaled_images:
    print(calculate_center_of_mass(image))

### Trasladar

In [ ]:
def translate(image, x, y):
    # Definir una imagen binarizada de ejemplo
    imagen_original = image

    # Crear un nuevo arreglo de numpy relleno con ceros
    filas, columnas = imagen_original.shape
    imagen_traslada = np.zeros((filas, columnas), dtype=np.uint8)

    # Copiar los valores de la imagen original en la posición correspondiente en el nuevo arreglo
    for i in range(filas):
        for j in range(columnas):
            if i + y < filas and j + x < columnas:
                imagen_traslada[i + y, j + x] = imagen_original[i, j]

    # Mostrar la imagen original y la imagen trasladada
    return imagen_traslada

translated_images = []

for image in scaled_images:
    Xcm, Ycm = calculate_center_of_mass(image)
    translated_images.append(translate(image, int(image.shape[1]/2 - Xcm),
                                              int(image.shape[0]/2 - Ycm)))

showImages(scaled_images)
showImages(translated_images)

### Momentos centrales

In [ ]:
for i in range(len(scaled_images)):
    print('Imagen original', i + 1)
    for p in range(0, 3):
        q_values = []
        for q in range(0, 3):
            result = central_moment(scaled_images[i], p, q)
            q_values.append(result)
        print(q_values)
    print('Imagen transladada', i + 1)
    for p in range(0, 3):
        q_values = []
        for q in range(0, 3):
            result = central_moment(translated_images[i], p, q)
            q_values.append(result)
        print(q_values)
    print()

### 8. Para cada objeto, realizar una rotación* con un ángulo x, y calcular los tres primeros momentos de Hu, antes y después de la rotación 

$$
\begin{matrix}
\varphi_1 = \mu_{20} + \mu_{02} \\
\varphi_2 = (\mu_{20} - \mu_{02})^2 + 4\mu_{11}^2 \\
\varphi_3 = (\mu_{30} - 3\mu_{12})^2 + (3\mu_{21} - \mu_{03})^2
\end{matrix}
$$

> *Nota: para rotar los objetos binarios, aplicar la matriz de rotación vista en clase. Usar interpolación bilineal o algún filtro para rellenar los huecos que hayan quedado al rotar


In [ ]:
import numpy as np

def rotate_binary_image(image, angle_rad):
    # Obtener las dimensiones de la imagen original
    height, width = image.shape
    
    # Calcular las dimensiones de la imagen de destino
    cos_theta = np.cos(angle_rad)
    sin_theta = np.sin(angle_rad)
    new_width = int(abs(width * cos_theta) + abs(height * sin_theta))
    new_height = int(abs(height * cos_theta) + abs(width * sin_theta))

    # Crear la matriz vacía de la imagen de destino
    rotated_image = np.zeros((height, width), dtype=np.uint8)

    # Calcular el centro de la imagen original
    cx = width // 2
    cy = height // 2

    # Iterar sobre cada píxel en la imagen de destino
    for x in range(width):
        for y in range(height):
            # Calcular las coordenadas del píxel en la imagen original
            tx = x - width // 2
            ty = y - height // 2
            px = int(tx * cos_theta - ty * sin_theta) + cx
            py = int(tx * sin_theta + ty * cos_theta) + cy

            # Copiar el valor del píxel de la imagen original a la imagen de destino
            if (px >= 0 and px < width and py >= 0 and py < height):
                rotated_image[y,x] = image[py,px]

    return rotated_image

rotatedImages = []
angle = 0.78539816

for image in translated_images:
    rotatedImages.append(rotate_binary_image(image,angle))

showImages(rotatedImages)

### Momentos de hu

In [ ]:
def calculate_moments(image):
    # Obtener las dimensiones de la imagen
    height, width = image.shape

    # Calcular los momentos de orden cero
    m00 = np.sum(image)

    # Calcular las coordenadas del centro de masa
    xcm = np.sum(np.multiply(np.arange(width), np.sum(image, axis=0))) / m00
    ycm = np.sum(np.multiply(np.arange(height), np.sum(image, axis=1))) / m00

    # Calcular los momentos centrales
    mu20 = central_moment(image,2,0)
    mu02 = central_moment(image,0,2)
    mu11 = central_moment(image,1,1)

    # Calcular los momentos normalizados
    nu20 = normalized_moments(image,2,0)
    nu02 = normalized_moments(image,0,2)
    nu11 = normalized_moments(image,1,1)
    nu30 = normalized_moments(image,3,0)
    nu12 = normalized_moments(image,1,2)
    nu21 = normalized_moments(image,2,1)
    nu03 = normalized_moments(image,0,3)

    # Calcular los momentos de Hu
    phi1 = nu20 + nu02
    phi2 = np.power(nu20 - nu02, 2) + 4 * np.power(nu11, 2)
    phi3 = np.power(nu30 - 3 * nu12, 2) + np.power(3 * nu21 - nu03, 2)
    return phi1, phi2, phi3


for i in range(len(translated_images)):
    print("Imagen antes de rotar", i + 1)
    print(calculate_moments(translated_images[i]))
    print("Imagen despues de rotar", i + 1)
    print(calculate_moments(rotatedImages[i]))
    print()


### 9. Calcular los ejes principales de cada uno de los objetos y alinearlos en una misma dirección, por ejemplo, el eje Y

In [ ]:
def moment_matrix(image):
    return np.array([[central_moment(image, 2, 0), central_moment(image, 1, 1)],
            [central_moment(image, 1, 1), central_moment(image, 0, 2)]])

def align_to_x_axis(moment_mat):
    # Calculate the moments of the image with respect to the centroid
    u20 = moment_mat[0, 0]
    u11 = moment_mat[0, 1]
    u02 = moment_mat[1, 1]
    
    # Add a small epsilon value to avoid division by zero
    eps = 1e-8

    # Calculate the angle to align with the X axis
    phi = 0.5 * np.arctan2(2 * u11, u20 - u02 + eps)

    return phi

second_rotated_images = []

for image in translated_images:
    second_rotated_images.append(rotate_binary_image(image, align_to_x_axis(moment_matrix(image))))

showImages(second_rotated_images)

### 10. Realizar una superposición entre cada pareja de objetos haciendo coincidir el centro de masa. En una tabla de 10 × 10 indicar el número de pixeles comunes (𝑃𝑐) y pixeles no comunes, 𝑃+ y 𝑃−, respectivamente, entre los 10 objetos. 

In [ ]:
common_pixels = np.zeros((len(rotatedImages), len(rotatedImages)))

for i in range(len(rotatedImages)):
    for j in range(len(rotatedImages)):
        intersection = np.logical_and(rotatedImages[i], rotatedImages[j])
        common_pixels[i,j] = np.sum(intersection)

print(common_pixels)


In [ ]:
P_plus = []
P_minus = []

for i in range(len(common_pixels)):
    P_Plus_i = []
    P_Minus_i = []
    for j in range(i, len(common_pixels)):
        P_Plus_i.append(int(common_pixels[i][i] - common_pixels[i][j]))
        P_Minus_i.append(int(common_pixels[j][j] - common_pixels[i][j]))
    P_plus.append(P_Plus_i)
    P_minus.append(P_Minus_i)

print("P+")
tabla_pixeles = pd.DataFrame.from_records(P_plus)
display(tabla_pixeles)
print()
print("P-")
tabla_pixeles = pd.DataFrame.from_records(P_minus)
display(tabla_pixeles)

### 11. Con el Algoritmo Húngaro, mover los pixeles 𝑃+ a los pixeles 𝑃− y realizar una tabla de las distancias mínimas obtenidas al comparar cada pareja de objetos.

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# Ejemplo de matriz de costos
cost_matrix = np.array([[9, 7, 8, 6],
                       [6, 5, 3, 2],
                       [8, 8, 6, 8],
                       [10, 6, 9, 5]])

# Función para resolver el problema de asignación utilizando el algoritmo húngaro
def hungarian_algorithm(cost_matrix):
    # Encontrar la asignación óptima de costos
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    total_cost = cost_matrix[row_ind, col_ind].sum()
    return row_ind, col_ind, total_cost

# Llamar a la función y obtener los resultados
row_ind, col_ind, total_cost = hungarian_algorithm(cost_matrix)

# Imprimir los resultados
print("Asignación óptima:")
for i in range(len(row_ind)):
    print("Tarea {} asignada a Trabajador {}".format(row_ind[i], col_ind[i]))
print("Costo total de asignación: {}".format(total_cost))


In [ ]:
def min_distance(img1, img2):
    p_plus = np.argwhere(np.logical_and(img1 == 1, img2 == 0))  # Pixeles P+
    p_minus = np.argwhere(np.logical_and(img1 == 0, img2 == 1))  # Pixeles P-

    n_plus = len(p_plus)
    n_minus = len(p_minus)

    if n_plus == 0 or n_minus == 0:
        return 0

    # Calculamos la matriz de costos
    cost_matrix = np.zeros((n_plus, n_minus))
    for i in range(n_plus):
        for j in range(n_minus):
            cost_matrix[i][j] = np.linalg.norm(p_plus[i] - p_minus[j])

    # Resolvemos el problema de asignación usando el algoritmo húngaro
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # Retornamos la suma de los costos mínimos
    return cost_matrix[row_ind, col_ind].sum()

costs = np.zeros((10, 10))

for i in range(len(second_rotated_images)):
    for j in range(i, (len(second_rotated_images))):
        costs[i, j] = int(min_distance(second_rotated_images[i], second_rotated_images[j]))

for cost in costs:
    print(list(cost))

## 12. Analizar cada uno de los resultados y obtener conclusiones. En las conclusiones aclarar si las ecuaciones (1), (2) y (3) son invariantes ante dichas transformaciones y si el método de escalamiento, alineación por ejes principales y aplicación del Algoritmo Húngaro para mover los pixeles, es adecuado para dar una medida de similitud.

### 1. En el primer ejercicio los 1 pixeles de una imagen binaria se calculan contando la cantidad de pixeles que tienen el valor de 1 dentro de la misma imagen y estos pixeles representan un objeto o una region en la imagen.

### 2. La transformacion de escala es una operacion que realiza un cambio en el tamaño de una imagen. En este caso en donde las imagenes son binarias, fue posible realizar una transformacion utilizando algoritmos de interpolacion 

### 3. La invariante de una imagen es una medida que es utilizada al momento de comparar imagenes y asi se determina si las imagenes son similares o no

### 4. Los graficos de celdas de resolucion de 1 pixel pueden ser obtenidos al crear un archivo con resolucion de 1 pixel y luego dividiendo el archivo en celdas individuales para luego utilizar esta division para los graficos de celdas

### 5. La conectividad usando la vecindad 8 es la tecnica utilizada en el procesamiento de imagenes para asi determinar la relacion entre los pixeles adyacentes en una imagen

### 6. El centro de masa es el punto en el cual se considera que toda la masa de un objeto se concentra en este punto, la traslacion es el movimiento de un objeto o una imagen de un lugar a otro sin rotacion y el momento central es la medida de la distribucion de la masa de un objeto o imagen en relacion a su centro de masa 

### 7. Los momentos hu son un conjunto de momentos invariantes de una imagen digital en escala de grises y se utilizan en la vision de una computadora y en la extraccion de caracteristicas de una imagen

### 8. La alineacion de las imagenes implica ajustar y superponer las imagenes para que esten en la misma direccion que un objeto o imagen de referencia comun.

### 9. En la superposicion de imagenes se realiza la edicion entre dos imagenes para hacer una sola imagen, en esta nueva imagen se da la ilusion de que las imagenes se superponen

### 10. El algoritmo hungaro se utiliza para encontrar la asignacion optima entre dos conjuntos de elementos